In [2]:
import subprocess
import glob
import cv2
import os
import numpy as np
import random
import shutil

In [17]:
INPUT_FILE = "D:\\Workspaces\\malexandersalazar\\tools-python-quick-video-noise-reducer\\src\\resources\\lyra-iorek.mp4"
NOISE_TYPE = "saltpepper"

In [18]:
def convertFPS(input_file,fps,output_file):
    parameters = {}
    parameters["input_file"] = input_file
    parameters["filter"] = "fps=" + str(fps)
    parameters["output_file"] = output_file

    commands_list = [
        "ffmpeg",
        "-i",
        parameters["input_file"],
        "-filter:v",
        parameters["filter"],
        parameters["output_file"]
        ]

    return commands_list

In [19]:
def extractFrames(input_file, frames_directory):
    parameters = {}
    parameters["rate_1"] = "1"
    parameters["input_file"] = input_file
    parameters["rate_2"] = "1"
    parameters["output_file"] = frames_directory + "\\%04d.jpg"

    commands_list = [
        "ffmpeg",
        "-r",
        parameters["rate_1"],
        "-i",
        parameters["input_file"],
        "-r",
        parameters["rate_2"],
        parameters["output_file"]
        ]

    return commands_list

In [20]:
def apply_noise(noise_type,image):
    if noise_type == "saltpepper":
        prob = 0.1
        output = np.zeros(image.shape,np.uint8)
        thres = 1 - prob 
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                rdn = random.random()
                if rdn < prob:
                    output[i][j] = 0
                elif rdn > thres:
                    output[i][j] = 255
                else:
                    output[i][j] = image[i][j]
        return output
    elif noise_type =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy

In [21]:
def join_frames_to_video(input_files,output_directory,file_name_without_ext):
    parameters = {}
    parameters["framerate"] = "24"
    parameters["input_files"] = input_files
    parameters["video_codec"] = "libx264"
    parameters["output_rate"] = "24"
    parameters["pixel_format"] = "yuv420p"
    parameters["output_file"] = output_directory + "\\" + file_name_without_ext + "_noise_noaudio.mp4"

    commands_list = [
        "ffmpeg",
        "-framerate",
        parameters["framerate"],
        "-i",
        parameters["input_files"],
        "-c:v",
        parameters["video_codec"],
        "-r",
        parameters["output_rate"],
        "-pix_fmt",
        parameters["pixel_format"],
        parameters["output_file"]
        ]

    return commands_list

In [22]:
owd = os.getcwd()

current_img_dir = os.path.dirname(INPUT_FILE)
input_filename = os.path.basename(INPUT_FILE)
input_filename_without_ext = os.path.splitext(input_filename)[0]
new_working_dir = current_img_dir + "\\" + input_filename_without_ext
os.mkdir(new_working_dir)
os.chdir(new_working_dir)

In [23]:
base_file = "24fps.mp4"
subprocess.run(convertFPS(INPUT_FILE,24,base_file))

frames_directory = new_working_dir + "\\frames"
os.mkdir(frames_directory)
subprocess.run(extractFrames(base_file, frames_directory))

CompletedProcess(args=['ffmpeg', '-r', '1', '-i', '24fps.mp4', '-r', '1', 'D:\\Workspaces\\malexandersalazar\\tools-python-quick-video-noise-reducer\\src\\resources\\lyra-iorek\\frames\\%04d.jpg'], returncode=0)

In [24]:
altered_frames_directory = new_working_dir + "\\altered_frames"
os.mkdir(altered_frames_directory)

for img_file in glob.glob(frames_directory + "\\*.jpg"):
    file_name = os.path.basename(img_file)
    file_name_without_ext = os.path.splitext(file_name)[0]

    img = cv2.imread(img_file, flags=cv2.IMREAD_COLOR)

    new_img = apply_noise(NOISE_TYPE,img)
    
    new_file_name = altered_frames_directory + '\\' + file_name_without_ext + '.jpg' 
    cv2.imwrite(new_file_name, new_img)

In [25]:
subprocess.run(join_frames_to_video(altered_frames_directory + "\\%04d.jpg", current_img_dir, input_filename_without_ext))

CompletedProcess(args=['ffmpeg', '-framerate', '24', '-i', 'D:\\Workspaces\\malexandersalazar\\tools-python-quick-video-noise-reducer\\src\\resources\\lyra-iorek\\altered_frames\\%04d.jpg', '-c:v', 'libx264', '-r', '24', '-pix_fmt', 'yuv420p', 'D:\\Workspaces\\malexandersalazar\\tools-python-quick-video-noise-reducer\\src\\resources\\lyra-iorek_noise_noaudio.mp4'], returncode=0)

In [26]:
shutil.rmtree(new_working_dir, ignore_errors=True)

In [27]:
os.chdir(owd)
os.rmdir(new_working_dir)

In [28]:
os.chdir(current_img_dir)
subprocess.run("ffmpeg -i " + input_filename_without_ext + ".mp4 -vn " + input_filename_without_ext + ".mp3")
subprocess.run("ffmpeg -i " + input_filename_without_ext + "_noise_noaudio.mp4 -i " + input_filename_without_ext + ".mp3 -c:v copy -map 0:v -map 1:a -y " + input_filename_without_ext + "_noisy.mp4 ")

os.remove(input_filename_without_ext + "_noise_noaudio.mp4")
os.remove(input_filename_without_ext + ".mp3")

CompletedProcess(args='ffmpeg -i lyra-iorek_noise_noaudio.mp4 -i lyra-iorek.mp3 -c:v copy -map 0:v -map 1:a -y lyra-iorek_noise_audio.mp4 ', returncode=0)